In [59]:
import pandas as pd
import os
import numpy as np
print("Current directory:", os.getcwd())
import sys
sys.path.append('.')
from MDQN import M_DQN_Agent
import torch
from collections import deque
from tqdm import tqdm
from Trading_Simulator import TradingEnvironment, pca_res_gen, fourier_signal_extractor

%reload_ext autoreload
%autoreload 2

Current directory: /Users/quintengeerts/Desktop/master thesis/stat_arb_thesis/empirical_analysis


In [7]:
import os
filename = 'price_df.csv'
if os.path.exists(filename):
    price_df = pd.read_csv(filename)
    price_df = price_df.rename(columns={'Unnamed: 0':'date'})
    price_df.set_index('date', inplace=True)
    price_df.index = pd.to_datetime(price_df.index)

returns = price_df.diff(1)

In [ ]:
action_to_portfolio = {0:-1, 1:0, 2: 1}


In [67]:
# PARAMETERS
seed = 100
#seed = np.random.randint(0,100000)

#writer = SummaryWriter("runs/"+"DQN_LL_new_1")
frames = 5000
BUFFER_SIZE = 1000000
BATCH_SIZE = 64
GAMMA = 0.99
TAU = 1e-2
eps_frames=5000
min_eps=0.025
LR = 1e-3
UPDATE_EVERY = 1
n_step = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using ", device)
eps_fixed = False
SAVE_MODEL = False
file_name = 'N1'

# Set seeds
#env.seed(seed)
#nv.action_space.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# action_size     = env.action_space.n #### going to fix this
action_size       = 3
#state_size        = env.observation_space.shape
state_size        = [59]

agent = M_DQN_Agent(state_size=state_size,    
                        action_size=action_size,
                        layer_size=64,
                        BATCH_SIZE=BATCH_SIZE, 
                        BUFFER_SIZE=BUFFER_SIZE, 
                        LR=LR, 
                        TAU=TAU, 
                        GAMMA=GAMMA, 
                        UPDATE_EVERY=UPDATE_EVERY, 
                        device=device, 
                        seed=seed)

##########################
action_to_portfolio = {0:-1, 1:0, 2: 1}
scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores
output_history = []
frame = 0
if eps_fixed:
    eps = 0
else:
    eps = 1
eps_start = 1
i_episode = 1

env = TradingEnvironment(financial_dataset=price_df.dropna(axis=0, thresh=300),
                        residual_generator=pca_res_gen,
                        signal_generator=fourier_signal_extractor,
                        episode_length=100,
                        lookback_window=252,
                        signal_window=60,
                        transaction_costs=0.0,
                        short_cost=0.0)

state, _ = env.warm_up()
state_vec= state[env.tradeable_tickers].values.T # turns the dataframe for the requested tickers into a ndarray

score = 1

for frame in tqdm(range(1, frames+1)):
    action = agent.act_para(state_vec, eps)
    portfolio_allocations = np.vectorize(action_to_portfolio.get)(action) #actions need to be turned into allocations
    next_state, reward, done, info = env.step(portfolio_allocations.T)
    

    for idx, ticker in enumerate(env.old_tradeable_tickers):
        if done[ticker].values[0]:
            # if the ticker is removed from the tradeable tickers, done = true and new_state is the last one
            agent.step(state[ticker].values, action[0,idx], reward[ticker].values[0], state[ticker].values, True)
        else:
            # if the ticker is not removed from the tradeable tickers, done = false 
            agent.step(state[ticker].values, action[0,idx], reward[ticker].values[0], next_state[ticker].values, False)
    
    state     = next_state
    state_vec = next_state[env.tradeable_tickers].values.T

    score *= (env.p_l + 1)
    # linear annealing to the min epsilon value until eps_frames and from there slowly decease epsilon to 0 until the end of training
    if eps_fixed == False:
        if frame < eps_frames:
            eps = max(eps_start - (frame*(1/eps_frames)), min_eps)
        else:
            eps = max(min_eps - min_eps*((frame-eps_frames)/(frames-eps_frames)), 0.001)
    
    if frame % env.ep_N == 0:
        scores_window.append(score-1)       # save most recent score
        scores.append(score-1)              # save most recent score
        #writer.add_scalar("Average100", np.mean(scores_window), frame)
        output_history.append(np.mean(scores_window))
        env.evaluate_performance()
        print('\rEpisode {}\tFrame {} \tAverage Score: {:.2f}'.format(i_episode, frame, np.mean(scores_window)), end="")
        #if i_episode % 100 == 0:
            #print('\rEpisode {}\tFrame {}\tAverage Score: {:.2f}'.format(i_episode,frame, np.mean(scores_window)))
        i_episode +=1 
        score = 1 

Using  cpu


  2%|▏         | 100/5000 [01:44<1:37:21,  1.19s/it]

 Episode: 0 -- Annualized Return: 1.4% -- Annualized Volatility: 2.81% -- Annualized Sharpe: 0.5
 Episode: 0 -- Total ann. Return: 1.4% -- Total ann. Volatility: 2.81% -- Total ann. Sharpe: 0.5
Episode 1	Frame 100 	Average Score: 0.01

  4%|▍         | 200/5000 [03:45<1:42:50,  1.29s/it]

 Episode: 1 -- Annualized Return: 11.02% -- Annualized Volatility: 2.51% -- Annualized Sharpe: 4.39
 Episode: 1 -- Total ann. Return: 6.1% -- Total ann. Volatility: 2.68% -- Total ann. Sharpe: 2.28
Episode 2	Frame 200 	Average Score: 0.02

...

 26%|██▌       | 1300/5000 [36:59<2:32:46,  2.48s/it]

 Episode: 12 -- Annualized Return: 3.68% -- Annualized Volatility: 1.84% -- Annualized Sharpe: 2.0
 Episode: 12 -- Total ann. Return: 2.16% -- Total ann. Volatility: 2.94% -- Total ann. Sharpe: 0.73
Episode 13	Frame 1300 	Average Score: 0.01

 28%|██▊       | 1400/5000 [41:11<2:33:31,  2.56s/it]

 Episode: 13 -- Annualized Return: 11.21% -- Annualized Volatility: 4.31% -- Annualized Sharpe: 2.6
 Episode: 13 -- Total ann. Return: 2.78% -- Total ann. Volatility: 3.06% -- Total ann. Sharpe: 0.91
Episode 14	Frame 1400 	Average Score: 0.01

 30%|██▉       | 1480/5000 [44:42<2:38:08,  2.70s/it]

In [65]:
output_history

[0.005547126501087352,
 0.023947481263181802,
 0.0087867380036832,
 0.006203484241557006,
 0.001863903434750891,
 0.008111305862654794,
 0.0036458193807671186,
 0.00914485225259061,
 0.008006006939638032,
 0.009368546316381265]